In [ ]:
import requests
import zipfile
import duckdb
import os
import time

# URL of the ZIP file
url1 = "https://www.stats.govt.nz/assets/Uploads/Business-employment-data/Business-employment-data-March-2024-quarter/Download-data/business-employment-data-march-2024-quarter.zip"
url2 = "https://www.stats.govt.nz/assets/Uploads/Business-financial-data/Business-financial-data-March-2024-quarter/Download-data/business-financial-data-march-2024.zip"

local_zip_path1 = './business_employment_data.zip'
local_zip_path2 = './business_financial_data.zip'
extracted_dir = './extracted/'

csv_file_name1 = 'machine-readable-business-employment-data-mar-2024-quarter.csv'
csv_file_name2 = 'business-financial-data-march-2024-csv.csv'
csv_file_path1 = os.path.join(extracted_dir, csv_file_name1)
csv_file_path2 = os.path.join(extracted_dir, csv_file_name2)

# Download zip file 1 and Extract
response = requests.get(url1)
with open(local_zip_path1, 'wb') as f:
    f.write(response.content)

with zipfile.ZipFile(local_zip_path1, 'r') as zip_ref:
    # Extract all files
    zip_ref.extractall(extracted_dir)

# Download zip file 2 and Extract
response = requests.get(url2)
with open(local_zip_path2, 'wb') as f:
    f.write(response.content)

with zipfile.ZipFile(local_zip_path2, 'r') as zip_ref:
    # Extract all files
    zip_ref.extractall(extracted_dir)

# create Duckdb Database
conn = duckdb.connect("employment.db")

create_table_query1 = f"""
CREATE OR REPLACE TABLE  business_employment AS
SELECT * FROM read_csv_auto('{csv_file_path1}');
"""

create_table_query2 = f"""
CREATE OR REPLACE TABLE business_financial AS
SELECT * FROM read_csv_auto('{csv_file_path2}');
"""

# Create Tables
conn.execute(create_table_query1)
conn.execute(create_table_query2)

conn.sql ("SHOW ALL TABLES").show()
cur_time = time.time()
conn.sql("""
    select Series_title_2,avg(data_value) from business_employment where Series_title_2 IN (
    SELECT DISTINCT Series_title_2
    FROM business_financial where Series_title_2 not IN
    ( SELECT DISTINCT Series_title_2
    FROM business_financial
    WHERE Series_title_1 like 'Salaries and wages' and Period IN (2016.03, 2016.06, 2016.09, 2016.12) ) )
    and Series_title_1 like 'Filled jobs' and Series_title_3 like 'Actual'
    group by Series_title_2
"""
).show()
print(f"time1: {time.time() - cur_time}")

cur_time = time.time()
conn.sql("""
-- Step 1: Identify distinct industries that have 'Salaries and wages' data
-- for the specified periods (Q1 to Q4 2016).
WITH DistinctIndustries AS (
    SELECT DISTINCT Series_title_2
    FROM business_financial
    WHERE Series_title_1 LIKE 'Salaries and wages'
      AND Period IN (2016.03, 2016.06, 2016.09, 2016.12)
),

-- Step 2: Filter out industries that do not have 'Salaries and wages' data
-- for the specified periods (Q1 to Q4 2016).
FilteredIndustries AS (
    SELECT DISTINCT Series_title_2
    FROM business_financial
    WHERE Series_title_2 NOT IN (SELECT Series_title_2 FROM DistinctIndustries)
      AND Series_title_1 LIKE 'Salaries and wages'
      AND Period NOT IN (2016.03, 2016.06, 2016.09, 2016.12)
),

-- Step 3: Select relevant data from 'business_employment' for the filtered industries
-- that have 'Filled jobs' and 'Actual' data.
FilteredData AS (
    SELECT Series_title_2, Data_value
    FROM business_employment
    WHERE Series_title_2 IN (SELECT Series_title_2 FROM FilteredIndustries)
      AND Series_title_1 LIKE 'Filled jobs'
      AND Series_title_3 LIKE 'Actual'
)

-- Step 4: Calculate the average 'Data_value' for each industry.
SELECT Series_title_2, AVG(Data_value) AS avg_data_value
FROM FilteredData
GROUP BY Series_title_2;

""").show()
print(f"time2: {time.time() - cur_time}")


┌────────────┬─────────┬─────────────────────┬──────────────────────┬──────────────────────────────────────┬───────────┐
│  database  │ schema  │        name         │     column_names     │             column_types             │ temporary │
│  varchar   │ varchar │       varchar       │      varchar[]       │              varchar[]               │  boolean  │
├────────────┼─────────┼─────────────────────┼──────────────────────┼──────────────────────────────────────┼───────────┤
│ employment │ main    │ business_employment │ [Series_reference,…  │ [VARCHAR, DOUBLE, DOUBLE, VARCHAR,…  │ false     │
│ employment │ main    │ business_financial  │ [Series_reference,…  │ [VARCHAR, DOUBLE, DOUBLE, VARCHAR,…  │ false     │
└────────────┴─────────┴─────────────────────┴──────────────────────┴──────────────────────────────────────┴───────────┘

┌─────────────────────────────────┬────────────────────┐
│         Series_title_2          │  avg(data_value)   │
│             varchar             │   